# Basic wowp usage

## Turning functions into actors

### 1. Define a function

We will define a simple `times2` function. Annotations (see [PEP 3107](https://www.python.org/dev/peps/pep-3107/)) will be used for output port names.

In [2]:
def times2(x) -> ('y'):
    '''Multiplies the input by 2
    '''
    return x * 2

### 2. Turn the function into an *actor*

In [3]:
from wowp.actors import FuncActor

times2_actor = FuncActor(times2)

### 3. Inspect actor's input and output ports

Input / output ports are accessible via `inports` and `outports` properties.

In [7]:
print('input  ports: {}'.format(times2_actor.inports.keys()))
print('output ports: {}'.format(times2_actor.outports.keys()))

input  ports: ['x']
output ports: ['y']


### 4. FuncActor is callable

In [10]:
x = 3
print('times2({}) = {}'.format(x, times2(x)))
print('times2_actor({}) = {}'.format(x, times2_actor(x)))
assert times2(x) == times2_actor(x)

times2(3) = 6
times2_actor(3) = 6


## Simple workflows

1. Workflows are created by connecting actor ports (input ports to output ports).
2. Ports get connected using the **`+=`** operator (`inport += outport`).

### Two actors chained together

Let's try something like `x -> actor1 -> actor2 -> out`.

In [23]:
# create two FuncActors
actor1 = FuncActor(lambda x: x * 2)
actor2 = FuncActor(lambda x: x + 1)

# chain the actors
# FuncActor output port is by default called out
actor2.inports['x'] += actor1.outports['out']

Provide an input value using the `put` method of input port(s).

In [25]:
# put an input value
x = 4
actor1.inports['x'].put(x)
# the workflow should have finished, now get the output
y = actor2.outports['out'].pop()
print('The result for x = {} is {}'.format(x, y))
# check the results
assert y == x * 2 + 1

The result for x = 4 is 9


## Creating a custom actor

In [11]:
from wowp import Actor

In [14]:
class StrActor(Actor):
    def __init__(self, *args, **kwargs):
        super(StrActor, self).__init__(*args, **kwargs)
        # specify input port
        self.inports.append('input')
        # and output ports
        self.outports.append('output')
    def on_input(self):
        # call fire if any input is available
        self.fire()
    def fire(self):
        self.outports['output'].put(str(self.inports['input'].pop()))

In [15]:
actor = StrActor(name='str_actor')

In [16]:
actor.inports['input'].put(123)

In [17]:
actor.outports['output'].pop()

'123'